In [55]:
import seaborn as sns
import numpy as np
import nibabel as nib
from nilearn import datasets, plotting
from statsmodels.stats.multitest import multipletests
from src.custom_plotting import plot_surface_stats, mkNifti
from nilearn import surface
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import gridspec
import itertools
from nilearn.plotting import plot_surf_roi

In [56]:
def fetch_surf(path):
    surf = dict()
    surf['infl_left'] = f'{path}/lh.inflated'#surface.load_surf_mesh()
    surf['infl_right'] = f'{path}/rh.inflated'#surface.load_surf_mesh()
    surf['pial_left'] = f'{path}/lh.pial'#surface.load_surf_mesh()
    surf['pial_right'] = f'{path}/rh.pial'#surface.load_surf_mesh()
    surf['sulc_left'] = f'{path}/lh.sulc'#surface.load_surf_data()
    surf['sulc_right'] = f'{path}/rh.sulc'#surface.load_surf_data()
    surf['sphere_left'] = f'{path}/lh.sphere'#surface.load_surf_mesh()
    surf['sphere_right'] = f'{path}/rh.sphere'#surface.load_surf_mesh()
    surf['curve_left'] = f'{path}/lh.curv'#surface.load_surf_data()
    surf['curve_right'] = f'{path}/rh.curv'#surface.load_surf_data()
    surf['area_left'] = f'{path}/lh.area'#surface.load_surf_data()
    surf['area_right'] = f'{path}/rh.area'#surface.load_surf_data()
    surf['thick_left'] = f'{path}/lh.thickness'#surface.load_surf_data()
    surf['thick_right'] = f'{path}/rh.thickness'#surface.load_surf_data()
    surf['white_left'] = f'{path}/lh.white'#surface.load_surf_mesh()
    surf['white_right'] = f'{path}/rh.white'#surface.load_surf_mesh()
    return surf

In [57]:
def load_noise_ceiling(mask_dir, sid, mask, dataset):
    noise_ceiling = np.load(f'{mask_dir}/sub-{sid}_space-T1w_desc-{dataset}-fracridge_stat-r_statmap.npy')
    noise_ceiling = noise_ceiling[mask]
    noise_ceiling[noise_ceiling <= 0.] = np.nan
    return noise_ceiling

In [58]:
def filter_r(rs, ps, p_crit=0.05, correct=True, threshold=True):
    if correct and (ps is not None):
        _, ps_corrected, _, _ = multipletests(ps, method='fdr_bh')
    elif not correct and (ps is not None):
        ps_corrected = ps.copy()
    else:
        ps_corrected = None

    if threshold and (ps is not None):
        rs[ps_corrected >= p_crit] = 0.
    else:
        rs[rs < 0.] = 0.
    return rs, ps_corrected

In [59]:
def full_model_mask(out_dir, sid, method):
    r = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_drop-None_single-None_method-{method}_rs.npy')
    p = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_drop-None_single-None_method-{method}_ps.npy')
    _, ps_corrected = filter_r(r, p)
    return ps_corrected >= 0.05

In [37]:
data_dir='/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/raw'
out_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim'
mask_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/Reliability'
top_fig_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/reports/figures/PlotVoxelEncoding'

fsaverage = datasets.fetch_surf_fsaverage()
cmap = sns.color_palette('magma', as_cmap=True)
roi_parcel = None
n_perm = 5000
n_subjs = 4
method = 'CV'

In [60]:
models = ['None']

In [66]:
for sid in ['02']:#['01', '02', '03', '04']:
    anat = nib.load(f'{data_dir}/anatomy/sub-{sid}/sub-{sid}_desc-preproc_T1w.nii.gz')
    img = nib.load(f'{mask_dir}/sub-{sid}_space-T1w_desc-test-fracridge_stat-r_statmap.nii.gz')
    mask = np.load(f'{out_dir}/Reliability/sub-{sid}_space-T1w_desc-test-fracridge_reliability-mask.npy').astype('bool')
    for model in models:
        r = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_drop-None_single-{model}_method-{method}_rs.npy')
        p = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_drop-None_single-{model}_method-{method}_ps.npy')
        r_filtered, _ = filter_r(r, p)
        volume = mkNifti(r_filtered, mask, img)
        # view = plotting.view_img_on_surf(volume, cmap=cmap_pred,
        #                                  symmetric_cmap=False, threshold=0.1,
        #                                  title=f'sub-{sid} model-{model}')

In [95]:
nib.save(volume, 'test.nii.gz')

In [91]:
view = plotting.view_img(volume, bg_img=anat,
                         cmap=cmap, black_bg=False,
                         symmetric_cmap=False,
                         threshold=1e-6)
view.save_as_html('volume.html')

In [94]:
surf = fetch_surf(f'{data_dir}/surfaces/sub-{sid}/surf')
hemi = 'right'
texture = surface.vol_to_surf(volume,
                              surf_mesh=surf[f'pial_{hemi}'],
                              interpolation='linear',
                              inner_mesh=surf[f'white_{hemi}'])
# nib.freesurfer.io.write_morph_data('test.gii', texture)

In [90]:
view = plotting.view_surf(surf_mesh=surf[f'infl_{hemi}'],
                          bg_map=surf[f'sulc_{hemi}'],
                          colorbar=False)
                          # surf_map=texture,
                          # threshold=1e-6)#, cmap=cmap)
                          # symmetric_cmap=False)
view.save_as_html('surface.html')